In [74]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [75]:
data = pd.read_csv('NASA_access_log.gz', sep='\n', header = None)

In [ ]:
r = re.compile('([^ ]+) - - \[(.+)\] \"(.+)\" (\d+) (\d+|-)')

m =  [r.match(x).groups() for x in data[0]]

In [72]:
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, "en_US.UTF-8")

hour = []
date = []
for i in xrange(len(m)):
    t = datetime.strptime(m[i][1][:20], "%d/%b/%Y:%H:%M:%S")
    d = t.replace(hour=0, minute=0, second=0)
    hour.append((t - d).total_seconds())
    date.append(d.strftime("%s"))


In [73]:
print hour[:5]
print date[:5]

[1.0, 6.0, 9.0, 11.0, 11.0]
['804549600', '804549600', '804549600', '804549600', '804549600']
